In [1]:
from hello.pid.lvpid import PIDController
from hello.pid.delay import *

In [3]:
from math import sin, pi

def rad_s(rpm):
    return rpm*2*pi/60


class AgMechanism():
    def __init__(self):
        self._hall_effect_loc = 280 * pi / 180  # approx
        self._hall_effect_radius = 10 * pi / 180


class AgProcess():
    def __init__(self, m=1/2.76, b=0, delay=3, op=0, pv=0, pamplitude=0.01, pperiod=30, ag_min=3):
        self.delay = DelayBuffer(delay, pv).cycle
        self.m = m
        self.b = b
        self.x = self.t = 0
        self.pamplitude = pamplitude
        self.pperiod = pperiod or 1
        self.pv = 0
        self.ag_min = ag_min

    def step(self, op):
        self.x += 1
        self.t += 1
        if self.x >= self.pperiod:
            self.x = 0
        m = self.m
        f = sin(self.x*2*pi/self.pperiod)*self.pamplitude + 1
        m *= f
        op = self.delay(op)
        self.pv = m*op + self.b
        if self.pv < self.ag_min:
            return 0
        return self.pv

    def step(self, op):
        self.pv = self.m * op + self.b
        pv = self.delay(self.pv)
        if pv < self.ag_min:
            return 0
        return self.pv

In [4]:
def ag_sim(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, per=180):
    pid = PIDController(p,(i),d,100,0)
    pid.man_to_auto(pv, sp, op)
    proc = AgProcess(delay=delay, pamplitude=amp, pperiod=per)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        if t < 15:
            pv = 0
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
    return data

In [5]:
%matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

Using matplotlib backend: TkAgg


In [6]:
def ag_sim2(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, per=180):
    global pid, proc
    pid = PIDController(p,i,d,100,0,0)
    pid.off_to_auto(pv, sp)
    proc = AgProcess(59/100, 0, delay=delay, pv=pv,pamplitude=0, pperiod=1)
    t = 0
    data = [(t, pv, op)]
    while True:
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
        t += 1
    return data

data = ag_sim2(.1, 0.008, 0, 180, 4, 0, 0, 12)
t, pv, op = list(zip(*data))
ax1.clear()
ax2.clear()
ax1.scatter(t, pv, label="PV")
ax2.scatter(t, op, label="OP")
ax1.set_xlim(0, 180)

ax2.set_xlim(0, 180)

(0, 180)

In [7]:
from pysrc.snippets import OptionCategory

class PIDOps(OptionCategory):
    p = 2
    i = 1
    d = 0
    beta = 0
    alpha = -1
    linearity = 1
    gamma = 0
    auto_max = 100
    auto_min = 10

class SimOps(OptionCategory):
    pid = PIDOps()
    minimum = 3
    

In [8]:
import sys
_g_axes = []
def mk_axes():
    global _g_axes
    _g_axes = []
    i = 1
    while True:
        try:
            ax = globals()['ax' + str(i)]
        except KeyError:
            break
        else:
            _g_axes.append(ax)
        i += 1

def axes():
    return _g_axes

In [9]:
def setup1(new=False):
    global fig, ax1, ax2, ax3, ax4, ax5, ax6
    
    if not plt.get_fignums() or new:
        fig = plt.figure()
        ax1 = fig.add_subplot(2,1,1)
        #ax5 = fig.add_subplot(3,2,5)
        #ax3 = fig.add_subplot(3,2,3)
        #ax4 = fig.add_subplot(3,2,4)
        ax2 = fig.add_subplot(2,1,2)
        #ax6 = fig.add_subplot(3,2,6)
        mk_axes()
        for a in ax1,: # ax3, ax5:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.8, b.height])
            a.grid()
        for a in ax2,: # ax4, ax6:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.8, b.height])
            a.grid()
    else:
        for a in axes():
            a.clear()
            a.grid()
        for t in fig.texts:
            t.remove()
    
    global colors, color
    colors = [
        "blue",
        "red",
        "green",
        "cyan",
        "purple",
        "orange",
        "black"
    ]

    import itertools
    color = itertools.cycle(colors).__next__
    


In [10]:
setup1()
def ag_sim2(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, ag_min=3):
    global pid, proc
    pid = PIDController(p,i,d,100,0,0)
    pid.off_to_auto(pv, sp)
    proc = AgProcess(55/100, 0, delay=delay, pv=pv,pamplitude=0, pperiod=1, ag_min=ag_min)
    t = 0
    data = [(t, pv, op)]
    while True:
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
        t += 1
    return data

data = ag_sim2(.1, 0.01, 0, 180, 4, 0, 0, 12)
t, pv, op = list(zip(*data))
ax1.clear()
ax2.clear()
ax1.scatter(t, pv, label="PV")
ax2.scatter(t, op, label="OP")
ax1.set_xlim(0, 180)

ax2.set_xlim(0, 180)

(0, 180)

In [11]:
n = int(32000/180)
sp = 20
t = 120
data = ag_sim2(.1, 0.01*n, 0, t*n, 0, 0, 0, sp, 0.05, 3)
data = [(a/n, b, c) for a,b,c in data]

In [18]:
fig = plt.figure()

In [19]:
x, y, z = list(zip(*data))
xa, ya = next((t,pv) for t, pv, op in data if pv >= (sp - 0.5))
xb = xa + 15
yb = next(pv for t,pv,op in data if t >= xb)

xc = xa + 60
yc = next(pv for t,pv,op in data if t >= xc)

fig.clear()
ax = fig.add_subplot(211)
ax2= fig.add_subplot(212)

ax.plot(x,y)
ax2.plot(x,y)

font = 'calibri'
thick_line = 0.75
thin_line = 0.5
thick_type = "--"
thin_type = ":"

ax.set_xlabel("Time (s)", fontname=font)
ax2.set_xlabel("Time (s)", fontname=font)
ax.set_ylabel("Ag PV (RPM)", fontname=font)
ax2.set_ylabel("Ag PV (RPM)", fontname=font)

xd = (xb+xc)/2

for a in ax, ax2:
    
    # hysteresis limits
    a.axhline(sp + 1, 0, t, color="black", linestyle = thick_type, linewidth=thick_line)   # A
    a.axhline(sp + 0.5, 0, t, color="black", linestyle = thin_type, linewidth=thin_line)   # B
    a.axhline(sp - 0.5, 0, t, color="black", linestyle = thin_type, linewidth=thin_line)   # C
    a.axhline(sp - 1, 0, t, color="black", linestyle = thick_type, linewidth=thick_line)   # D
    
    # time points
    a.axvline(xa, 0, sp+10, color="black", linestyle=thick_type, linewidth=thick_line)     # E
    a.axvline(xb, 0, sp+10, color="black", linestyle=thin_type, linewidth=thin_line)       # F
    a.axvline(xd, 0, sp+10, color="black", linestyle=thick_type, linewidth=thick_line)     # G
    a.axvline(xc, 0, sp+10, color="black", linestyle=thin_type, linewidth=thin_line)       # H
    
    
# annotations
ax2.annotate("A: SP + 1", (3, sp + 1.1), fontname=font)
ax2.annotate("B: SP + 0.5", (3, sp+.2), fontname=font)
ax2.annotate("C: SP - 0.5", (3, sp-.4), fontname=font)
ax2.annotate("D: SP - 1", (3, sp - 1.35), fontname=font)

ax2.annotate("E", (xa-3, sp + 1.7))
ax2.annotate("F", (xb-3, sp + 1.7))
ax2.annotate("G", (xd-3, sp + 1.7))
ax2.annotate("H", (xc-3, sp + 1.7))
       
ax.set_xlim(0, t)
ax.set_ylim(0, sp+10)
ax2.set_xlim(0, t)
ax2.set_ylim(sp - 2, sp + 2)

ax.xaxis.set_major_locator(plt.NullLocator())
ax2.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())
ax2.yaxis.set_major_locator(plt.NullLocator())

txt = """E: Time to SP - 0.5
F: E + 15
G: E + 60"""
txt = fig.text(0.14, 0.04, txt, fontname=font)

fig.show()